# Nonomino sudoku

### Idea: Given a (partially filled) 9 x 9 grid, we want to complete it with numbers from 1-9, respecting the following constraints:
     1) Only one digit per cell
     2) In any row, each of the digits from 1 to 9 appears only once
     3) In any column, each of the digits from 1 to 9 appears only once
     4) In any color group, each of the numbers from 1 to 9 appears only once
#### In the numpy file, there are two arrays of the following form:
     1) An array (9 x 9) which contains the grid with the pre-filled numbers. Empty boxes are represented by zeros. (board['sudokus'][i]['grid'])
     2) A second array (9 x 9) to indicate which color group a cell belongs to, using numbers from 1 to 9 (board['sudokus'][i]['group_grid'])

### ----------------------------------------------------------------------------------------------------------------------------------------------------------

### 1st Approach (Brute-force search technique)
#### This is a "proof-of-concept" solution based on backtracking that is quick to implement, but not optimized for performance - please see "2nd approach" further down for a more elegant solution. The idea is the following:
    1) Find the empty cells (value = 0) and assign a valid value (i.e. one respecting the above constraints) to it
    2) Repeat 1) until you would have to use a value that violates the constraints. In that case:
    3) Go back to the last cell you filled and try a different possible value. Then continue with 1) until 2) happens, in which  case you progressively increase the number of cells you revisit, and reassign possible values.
    4) Keep doing this until all the board is complete.
   

In [39]:
#Backtracking (Brute-force search technique), use the np.where function for faster access to the values
# function that is called to solve the sudoku
def SolveSudoku(bo,group):        
    i,j = np.where(bo==0)  #find the corresponding row and column of all empty cells
    if (i.size==0): #returns True where there are no more empty cells in the sudoku => it's solved
        return True
    else:
        #start with the 1st empty cell. Once it's filled, move on to the next empty cell
        row,column=i[0],j[0] 
    for i in range(1,10):
        if ValidNumber(bo,group,i,(row,column)):
            bo[row][column]=i #assign a number to the cell if it is valid
                
            if SolveSudoku(bo,group): #if the sudoku is solved 
                return True
            bo[row][column]=0 #else if is not solved, change the last value to 0 and repeat
    return False
     

#This function checks if the number that is suggested for an empty cell is valid according to the constrains described above
def ValidNumber(bo,group,num,pos):
    #check if this number already exist in the same row (except its own position)
    for j in range(len(bo)):
        if bo[pos[0]][j]==num and pos[1]!=j:
            return False
        
    #check if this number already exist in the same column
    for i in range(len(bo)):            
        if bo[i][pos[1]]==num and pos[0]!=i:
            return False
            
    #check if this number already exist in the same color group
    # use np.where() to find all tuples (i,j) which belong to the same group
    i,j=np.where(group==group[pos]) 
    for k in range(len(bo)):
        if bo[i[k]][j[k]]==num and (i[k],j[k])!=pos:
                return False
    return True

In [40]:
import numpy as np
import time
board = np.load('numpySudoku.npz') #load the data

for i in range(0,3): #solve the 3 sudoku
    bo=board['sudokus'][i]['grid']
    group=board['sudokus'][i]['group_grid']
    start = time.time()
    SolveSudoku(bo,group)
    end = time.time()
    print("Sudoku ",i+1,", time to run: {}".format(end - start),"\n" ,bo,"\n" )

Sudoku  1 , time to run: 9.073798179626465 
 [[3 5 8 1 9 6 2 7 4]
 [4 9 2 5 6 7 1 3 8]
 [6 1 3 9 7 8 4 2 5]
 [1 7 5 8 4 2 6 9 3]
 [8 2 6 4 5 3 7 1 9]
 [2 4 9 7 3 1 8 5 6]
 [9 8 7 3 2 4 5 6 1]
 [7 3 4 6 1 5 9 8 2]
 [5 6 1 2 8 9 3 4 7]] 

Sudoku  2 , time to run: 6.070539236068726 
 [[8 5 4 2 6 1 7 9 3]
 [3 7 1 9 8 2 4 5 6]
 [9 6 5 7 2 4 8 3 1]
 [7 1 3 6 4 9 2 8 5]
 [6 9 2 8 1 3 5 7 4]
 [2 4 8 5 7 6 3 1 9]
 [1 3 9 4 5 8 6 2 7]
 [5 2 6 3 9 7 1 4 8]
 [4 8 7 1 3 5 9 6 2]] 

Sudoku  3 , time to run: 2.2067346572875977 
 [[6 7 5 8 4 9 2 1 3]
 [4 9 2 1 3 8 6 7 5]
 [1 8 3 6 7 5 9 2 4]
 [2 5 7 4 1 6 3 9 8]
 [3 1 9 2 8 4 5 6 7]
 [9 3 4 5 6 7 1 8 2]
 [7 2 8 9 5 1 4 3 6]
 [5 6 1 7 2 3 8 4 9]
 [8 4 6 3 9 2 7 5 1]] 



### 2nd Approach
#### Now let's optimize the code and reduce runtime.

#### The idea is the following:
    1) Find the empty cells
    2) Create a dictionary (for this cell) which contains all the possible values that can go into the cell
    3) Start by filling those cells that have the smallest number of possible values
    4) Assign a value to the cell and remove this value (a) from the dictionary values of this cell and (b) from all the other dictionaries that share a common row, column or group with the newly filled cell.
    5) If the assigned value will not solve the sudoku, unfill the cell and update the possible values (i.e. revert (4))

In [41]:
# function that solves the sudoku with the help of the either functions
def SolveSudoku(bo,group):
    values = PossibleValues(bo,group)
    if len(values) == 0:
        return True #sudoku solved
    cell = min(values.keys(), key=lambda x: len(values[x])) #start with cells with the smallest number of possible values
    nums = values[cell]
    for num in nums:
        update = {cell: values[cell]}
        
        if FillCell(bo, cell, update,num,values,group):  
            if SolveSudoku(bo,group):  
                return True
        UnfillCell(bo,cell,update,values)  # invalid choice or didn't solve it => unfill the cell 
    return False
    

# function to find the possible values for the empty cells that respect the constrains
def PossibleValues(bo,group): 
    values={} #an empty dictionary for all the possible values in each empty cell
    numbers={n for n in range(1, 10)} # all the numbers from 1-9
    for i in range(len(bo)):
        for j in range(len(bo)):
            if bo[i][j]==0: #check if the cell is empty
                ans=set() #create a empty set
                for k in range(len(bo)):
                        if bo[k][j] != 0:
                            ans.add(bo[k][j]) #add in the set the numbers (non zero) that already exist in the same row
                        if bo[i][k] != 0:
                            ans.add(bo[i][k]) #add in the set the numbers (non zero) that already exist in the same column
                        m,n=np.where(group==group[i,j])
                        if bo[m[k]][n[k]]!=0:
                            ans.add(bo[m[k]][n[k]]) #add in the set the numbers (non zero) that already exist in the same group
                values[(i, j)] = numbers - ans # the values in this cell are the valid numbers that are not included in the ans set
              
   
    return values            
                        

#function that fills the cell with a certain number and updates the dictionary 'values'   
def FillCell(bo,cell,update,num,values,group):
    bo[cell[0]][cell[1]] = num #fill the cell with the selected number
    del values[cell] #delete the values for this key from the dictionary
    i, j = cell  #the 'coordinates' i,j of the cell 
    
    for k in values.keys():
        if num in values[k]:  
            if k[0] == i or k[1] == j or group[k[0],k[1]] == group[i, j]: #remove the number from dicts sharing the same row, column or group
                update[k] = num #keep the num in the update for unfilling
                values[k].remove(num)
                if len(values[k]) == 0:
                    return False
    return True

#function that unfill the cell and updates the dictionary 'values'
def UnfillCell(bo,cell,update,values):
    bo[cell[0]][cell[1]] = 0 #reassign the value 0 to the cell
    
    for k in update:
        if k not in values:
            values[k] = update[k]
        else:
            values[k].add(update[k])

In [42]:
import numpy as np
import time
#load the file
board = np.load('numpySudoku.npz')
for i in range(0,3): #solve the 3 sudoku
    bo=board['sudokus'][i]['grid']
    group=board['sudokus'][i]['group_grid']
    start = time.time()
    SolveSudoku(bo,group)
    end = time.time()
    print("Sudoku ",i+1,", time to run: {}".format(end - start),"\n" ,bo,"\n" )

Sudoku  1 , time to run: 1.7959978580474854 
 [[3 5 8 1 9 6 2 7 4]
 [4 9 2 5 6 7 1 3 8]
 [6 1 3 9 7 8 4 2 5]
 [1 7 5 8 4 2 6 9 3]
 [8 2 6 4 5 3 7 1 9]
 [2 4 9 7 3 1 8 5 6]
 [9 8 7 3 2 4 5 6 1]
 [7 3 4 6 1 5 9 8 2]
 [5 6 1 2 8 9 3 4 7]] 

Sudoku  2 , time to run: 0.4407472610473633 
 [[8 5 4 2 6 1 7 9 3]
 [3 7 1 9 8 2 4 5 6]
 [9 6 5 7 2 4 8 3 1]
 [7 1 3 6 4 9 2 8 5]
 [6 9 2 8 1 3 5 7 4]
 [2 4 8 5 7 6 3 1 9]
 [1 3 9 4 5 8 6 2 7]
 [5 2 6 3 9 7 1 4 8]
 [4 8 7 1 3 5 9 6 2]] 

Sudoku  3 , time to run: 0.4937169551849365 
 [[6 7 5 8 4 9 2 1 3]
 [4 9 2 1 3 8 6 7 5]
 [1 8 3 6 7 5 9 2 4]
 [2 5 7 4 1 6 3 9 8]
 [3 1 9 2 8 4 5 6 7]
 [9 3 4 5 6 7 1 8 2]
 [7 2 8 9 5 1 4 3 6]
 [5 6 1 7 2 3 8 4 9]
 [8 4 6 3 9 2 7 5 1]] 

